# Regresión logística

La regresión logística es un tipo de análisis de regresión utilizado para predecir el resultado de una variable categórica (una variable que puede adoptar un número limitado de categorías) en función de las variables independientes o predictoras.

The implementation of logistic regression in scikit-learn can be accessed from class LogisticRegression. This implementation can fit binary, One-vs- Rest, or multinomial logistic regression with optional L2 or L1 regularization.

In [1]:
# import libraries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../datasets/titanic_custom.csv', index_col=0)
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


## Tablas de contingencia

Estas tablas pueden resultar útiles a la hora de querer calcular probabilidades.

In [3]:
contingency_table = pd.crosstab(df['sex'], df['survived'])
contingency_table

survived,0,1
sex,,
female,127,339
male,682,161


In [4]:
total_sex = contingency_table.sum(axis = 1)
total_survived = contingency_table.sum(axis = 0)
print("Mujeres: {0} Hombres: {1}".format(total_sex[0], total_sex[1]))
print("Vivos: {0} Muertos: {1}".format(total_survived[0], total_survived[1]))

Mujeres: 466 Hombres: 843
Vivos: 809 Muertos: 500


## La probabilidad condicional

Define la probabilidad de que un cierto suceso sea verdad o no, sabiendo ciertas restricciones relacionadas con un evento que ya ha sucedido o va a suceder.

Ejemplo:

* ¿Cuál es la probabilidad de que un pasajero muera sabiendo que es un hombre?
* ¿Cuál es la probabilidad de que un pasajero muera sabiendo que es una mujer?


Para la primera pregunta:

$$
P(Died|Male) = \frac{Número\ total\ de\ muertes\ de\ hombres}{Número\ total\ de\ hombres} = \frac{Died \cap Male}{Male}
$$

In [5]:
# Probabilidad de que un hombre muera
p0 = contingency_table[0][1] / total_sex[1]
p0

0.8090154211150652

La probabilidad contraria:

In [6]:
# Probabilidad de que un hombre no muera
p01 = 1 - p0
p01

0.1909845788849348

$$
P(Died|Female) = \frac{Número\ total\ de\ muertes\ de\ mujeres}{Número\ total\ de\ mujeres} = \frac{Died \cap Female}{Female}
$$

In [7]:
# Probabilidad de que una mujer muera
p1 = contingency_table[0][0] / total_sex[0]
p1

0.27253218884120173

In [8]:
# Probabilidad de que una mujer no muera
p11 = 1 - p1
p11

0.7274678111587982

Este proceso se puede realizar de manera más cómoda:

In [9]:
prob = contingency_table.astype('float').div(contingency_table.sum(axis=1), axis = 0)
prob

survived,0,1
sex,,
female,0.272532,0.727468
male,0.809015,0.190985


## Ratio de probabilidades

Cociente entre los casos de éxito sobre los de fracaso en el suceso estudiado y para cada grupo. En el caso del *Titanic* tendríamos:

$P_{m}$: probabilidad de que muera sabiendo que es hombre

$P_{f}$: probabilidad de que muera sabiendo que es mujer

$$
odds_{Died\ |\ Male} = \frac{P_{m}}{1 - P_{m}} \\
odds_{Died\ |\ Female} = \frac{P_{f}}{1 - P_{f}}
$$

* Si $odds > 1$ : el numerador es mayor que el denominador. Por tanto la probabilidad de éxito es mayor que la probabilidad de fracaso.
* Si $odds < 1$: caso contrario
* Si $ odds = 1$ : éxito o fracaso son equiprobables $ p = 0.5$

In [11]:
odds_m = prob[0][1] / (1-prob[0][1])
odds_m

4.236024844720496

In [12]:
odds_f = prob[0][0] / (1 - prob[0][0])
odds_f

0.37463126843657824

Podemos realizar el cociente entre ratios. Esto nos dice cual de los dos ratios domina. Si es mayor que 1, domina el numerador, mientras que lo contrario significa que domina el denominador.

In [13]:
odds = odds_m / odds_f
odds

11.307184428033448

In [ ]:
df['sex'].value_counts()

In [14]:
pd.pivot_table(df, columns='sex', index='survived', values='age', aggfunc=np.mean)

sex,female,male
survived,,
0,25.255208,31.516412
1,29.815354,26.977778


In [15]:
pd.pivot_table(df, columns='sex', index='survived', values='fare', aggfunc=np.mean)

sex,female,male
survived,,
0,22.324084,23.545868
1,55.142048,37.189053


In [16]:
pd.pivot_table(df, columns='pclass', index='survived', values='fare', aggfunc=np.mean)

pclass,1,2,3
survived,,,
0,70.412533,19.801713,13.435451
1,98.023314,23.008124,12.916919
